In [1]:
import numpy as np
import torch
from torch import log, sqrt, exp
from scipy.stats import norm

Z—score Normalization的特点：
均值为0，标准差为1，适合无量纲比较

In [4]:
class Normalization:
    def __init__(self, mean_val=None, std_val=None):
        self.mean_val = mean_val
        self.std_val = std_val
    
    # 将x归一化--提高收敛速度和稳定性
    def normalize(self, x):
        return (x-self.mean_val)/self.std_val # z-score归一化


    
    # 反归一化
    def unnormalize(self, x):
        return x*self.std_val + self.mean_val
    

1.计算两个信号 x（参考信号）和y（估计信号）之间的相关性(corr) 平均绝对误差（MAP） 和平均绝对百分比误差 
$$ cov(x,y) = \frac{1}{n}\sum(x_i-\mu_x)(y_{i}-\mu_{y}) \tag{eq.1}$$


2.Pearson 相关系数
$$corr = \frac{cov(x,y)}{\sigma_{x}\sigma_{y}}([-1,1]) \tag{eq.2}$$

3.MAP(MAE, Mean Absolute Error) MAPE( Mean Absolute Error Percentage )
$$MAP=\frac{1}{n}\sum^{n}_{i=1}|x_{i}-y_{i}| \tag{eq.3}$$

4.MAPE(绝对百分比误差)
$$MAPE=\frac{1}{n}\sum^{n}_{i=1}|\frac{x_{i}-y_{i}}{x_{i}}| \tag{eq.4} $$


In [5]:
def metrics(y, x):
    #x: reference signal 参考信号
    #y: estimated signal 预测（估计）信号
    if torch.is_tensor(x):
        # 检查 x 是否位于GPU上
        if x.is_cuda:
            # 如果 x 位于GPU上，将其移动到CPU上
            # 因为numpy不支持GPU上的张量，必须将
            # 张量全部移动到CPU上 才能转化为NumPy数组
            x = x.cpu()
        x = x.numpy()
    if torch.is_tensor(y):
        if y.is_cuda:
            y = y.cpu()
        y = y.numpy()
    
    # correlation
    # keepdims=True 保持维度不变，避免降维
    x_mean = np.mean(x, axis=0, keepdims=True)
    y_mean = np.mean(y, axis=0, keepdims=True)
    x_std = np.std(x, axis=0, keepdims=True)
    y_std = np.std(y, axis=0, keepdims=True)
    corr = np.mean((x-x_mean)*(y-y_mean), axis=0, keepdims=True)/(x_std*y_std)
    
    # MAP
    map = np.mean(np.abs(x-y), axis=0, keepdims=True)

    # MAPE
    mape = np.mean(np.abs((x - y)/x), axis=0, keepdims=True)
    
    return torch.tensor(corr), torch.tensor(map), torch.tensor(mape)
        
    

$$B-S(call \ option) \\
spot--S \\
maturity--T \\
vol--sigma \\$$
$$C=S·N(d_1) - K·e^{-rT}· N(d_2)$$
$$\Phi(x)=\frac{1}{\sqrt{2\pi}}\int^{x}_{-\infty}e^{-\frac{t^2}{2}}dt $$
$$ d_{1}=\frac{\ln(S/K)+(r+\sigma^2/2)T}{\sigma\sqrt{T}} \tag{eq.5}$$

In [6]:
def call_option_pricer(spot, strike, maturity, r, vol):
    d1 = (log(spot / strike) + (r + 0.5 * vol * vol) * maturity) / (vol * sqrt(maturity))
    d2 = d1 - vol * sqrt(maturity)
    
    price = spot * norm.cdf(d1) - strike * exp(-r * maturity) * norm.cdf(d2)
    return price

B-S (put option)
$$P = K·e^{-rT}·N(-d_{2})- S·N(-d_1) \tag{eq.6}$$

In [7]:
def put_option_pricer(spot, strike, maturity, r, vol):
    d1 = (log(spot / strike) + (r + 0.5 * vol * vol) * maturity) / (vol * sqrt(maturity)) 
    d2 = d1 - vol * sqrt(maturity)
    
    price = -spot * norm.cdf(-d1) + strike * exp(-r*maturity) * norm.cdf(-d2)
    return price
    